Setup Workspace environment

In [ ]:
import json
import urllib.request
from urllib.parse import quote
from azureml.core import Workspace, Dataset
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Environment, BuildContext
from azureml.core.conda_dependencies import CondaDependencies
from azure.identity import DefaultAzureCredential
from azureml.core import Environment, Workspace,  runconfig, Experiment


# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your Azure Machine Learning workspace
subscription_id=""
resource_group=""
workspace=""
ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Datastore Download

In [ ]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = ''
resource_group = ''
workspace_name = ''

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='ADS')
#dataset.download(target_path='f', overwrite=False)

Convert COCO2JSONL

In [ ]:
!python3 path/to/coco2jsonl.py --input_coco_file_path "path/to/train/_annotations.coco.json" \
--output_dir "path/to/train/" --output_file_name "output_train.jsonl" \
--task_type "ObjectDetection" \
--base_url "path/to/.."

!python3 path/to/coco2jsonl.py --input_coco_file_path "path/to/valid/_annotations.coco.json" \
--output_dir "path/to/valid/" --output_file_name "output_valid.jsonl" \
--task_type "ObjectDetection" \
--base_url "path/to/.."

MLTABLE Creation

In [ ]:
from azure.ai.ml import automl
from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.constants import AssetTypes

my_training_data_input = Input(type=AssetTypes.MLTABLE, path="path/to/train/output_train.jsonl")


my_validation_data_input = Input(type=AssetTypes.MLTABLE, path="path/to/valid/output_valid.jsonl")


Training Definition

In [ ]:
from azure.ai.ml import automl

#compute_name = "gpu-cluster"
image_object_detection_job = automl.image_object_detection(
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label"
    #compute=compute_name,
    # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
    #outputs={"best_model": Output(type="mlflow_model")}
)

Auto Hyperparameter sweep

In [ ]:
image_object_detection_job.set_limits(max_trials=10, max_concurrent_trials=1)
#image_object_detection_job.set_training_parameters(model_name="yolov5")

Submit Job

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_object_detection_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")